# Projet C - Mini-GPT : Génération de Texte

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Construire et entraîner un petit modèle génératif type GPT

---

## Objectifs du projet

Dans ce projet, vous allez :
1. Comprendre la différence entre Encodeur (BERT) et Décodeur (GPT)
2. Implémenter le masque causal pour la génération
3. Entraîner un mini-GPT sur un corpus français
4. Générer du texte de manière autoregressive

> **📌 Note pédagogique** : Cette session introduit le **décodeur** (GPT) avec son **masque causal**. 
> C'est une architecture différente de l'encodeur (BERT) vu en Sessions 1-4.
>
> **Cross-attention** (mentionné en Session 2) : Dans les architectures **encodeur-décodeur** complètes (traduction, T5), 
> le décodeur utilise du cross-attention pour "interroger" l'encodeur. Ici, on se concentre sur le **décodeur seul** (GPT), 
> qui n'utilise que la **self-attention causale**.

## 0. Installation

In [ ]:
!pip install torch matplotlib numpy tqdm -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

torch.manual_seed(42)

---

## 1. Encodeur vs Décodeur : Quelle différence ?

### BERT (Encodeur) - Ce qu'on a fait jusqu'ici

```
Entrée:  "Le chat [MASK] sur le canapé"
                    ↓
         Chaque mot voit TOUS les autres
                    ↓
Sortie:  Prédire [MASK] = "dort"
```

- **Bidirectionnel** : chaque token voit le passé ET le futur
- **Usage** : Classification, NER, Question-Answering extractif

### GPT (Décodeur) - Ce qu'on fait maintenant

```
Entrée:  "Le chat dort sur le"
                    ↓
         Chaque mot voit SEULEMENT les précédents
                    ↓
Sortie:  Prédire le mot suivant = "canapé"
```

- **Unidirectionnel (causal)** : chaque token ne voit que le passé
- **Usage** : Génération de texte, chatbots, complétion

### Le masque causal

Pour empêcher un token de "tricher" en regardant le futur, on utilise un **masque causal** :

```
              Le   chat  dort  sur   le
      Le    [  1     0     0     0    0  ]   ← "Le" ne voit que lui-même
     chat   [  1     1     0     0    0  ]   ← "chat" voit "Le" et lui-même
     dort   [  1     1     1     0    0  ]   ← "dort" voit les 3 premiers
      sur   [  1     1     1     1    0  ]   ← etc.
       le   [  1     1     1     1    1  ]   ← dernier voit tout le passé
```

Les 0 deviennent `-∞` avant le softmax → attention = 0 sur ces positions.

---

## 2. Implémentation du Masque Causal

In [ ]:
# ============================================
# EXERCICE 1 : Créer le masque causal
# ============================================

def create_causal_mask(seq_len):
    """
    Crée un masque causal (triangulaire inférieur).
    
    Args:
        seq_len: Longueur de la séquence
    
    Returns:
        mask: Tensor booléen (seq_len, seq_len)
              True = position à masquer (ne pas regarder)
              False = position visible
    """
    # TODO: Créer une matrice triangulaire supérieure de True
    # Indice: torch.triu(torch.ones(...), diagonal=1)
    # diagonal=1 pour que la diagonale soit visible (un token se voit lui-même)
    
    mask = None  # À compléter
    
    return mask.bool()

# Test
mask = create_causal_mask(5)
print("Masque causal (True = masqué) :")
print(mask.int())  # Affiche 0 et 1 pour plus de lisibilité

In [ ]:
# Visualisation
plt.figure(figsize=(6, 5))
plt.imshow(~mask, cmap='Blues')  # ~ inverse pour afficher "visible" en bleu
plt.xticks(range(5), ['pos 0', 'pos 1', 'pos 2', 'pos 3', 'pos 4'])
plt.yticks(range(5), ['pos 0', 'pos 1', 'pos 2', 'pos 3', 'pos 4'])
plt.xlabel("Positions regardées (Keys)")
plt.ylabel("Positions qui regardent (Queries)")
plt.title("Masque Causal (bleu = visible)")
plt.colorbar(label="Visible")
plt.show()

---

## 3. Architecture Mini-GPT

On reprend notre Transformer et on ajoute le masque causal.

In [ ]:
class CausalSelfAttention(nn.Module):
    """Self-Attention avec masque causal (style GPT)."""
    
    def __init__(self, embed_dim, num_heads, max_len=512, dropout=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        # Pré-calculer le masque causal (enregistré comme buffer)
        mask = torch.triu(torch.ones(max_len, max_len), diagonal=1).bool()
        self.register_buffer('mask', mask)
    
    def forward(self, x):
        B, S, _ = x.shape
        
        # Projections Q, K, V
        Q = self.W_q(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        
        # Attention scores
        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.d_k)
        
        # Appliquer le masque causal
        scores = scores.masked_fill(self.mask[:S, :S], float('-inf'))
        
        # Softmax et dropout
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        # Output
        out = (attn @ V).transpose(1, 2).contiguous().view(B, S, self.embed_dim)
        return self.W_o(out)

In [ ]:
class GPTBlock(nn.Module):
    """Un bloc de GPT (attention causale + FFN)."""
    
    def __init__(self, embed_dim, num_heads, max_len=512, dropout=0.1):
        super().__init__()
        self.attn = CausalSelfAttention(embed_dim, num_heads, max_len, dropout)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(),
            nn.Linear(4 * embed_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = x + self.dropout(self.attn(self.norm1(x)))
        x = x + self.dropout(self.ff(self.norm2(x)))
        return x

In [ ]:
# ============================================
# EXERCICE 2 : Compléter le Mini-GPT
# ============================================

class MiniGPT(nn.Module):
    """
    Mini-GPT pour la génération de texte caractère par caractère.
    
    Args:
        vocab_size: Taille du vocabulaire (nombre de caractères uniques)
        embed_dim: Dimension des embeddings
        num_heads: Nombre de têtes d'attention
        num_layers: Nombre de blocs GPT
        max_len: Longueur maximale des séquences
        dropout: Taux de dropout
    """
    
    def __init__(self, vocab_size, embed_dim=128, num_heads=4, num_layers=4, 
                 max_len=256, dropout=0.1):
        super().__init__()
        
        self.max_len = max_len
        
        # Token embedding
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Position embedding (apprise, pas sinusoïdale)
        self.position_embedding = nn.Embedding(max_len, embed_dim)
        
        # Blocs GPT
        self.blocks = nn.ModuleList([
            GPTBlock(embed_dim, num_heads, max_len, dropout)
            for _ in range(num_layers)
        ])
        
        # Layer norm finale
        self.norm = nn.LayerNorm(embed_dim)
        
        # Tête de prédiction (projette vers le vocabulaire)
        self.head = nn.Linear(embed_dim, vocab_size)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        Args:
            x: Indices de tokens, shape (batch, seq_len)
        
        Returns:
            logits: Scores pour chaque token du vocabulaire, shape (batch, seq_len, vocab_size)
        """
        B, S = x.shape
        
        # TODO: Implémenter le forward
        
        # 1. Token embeddings
        tok_emb = None  # self.token_embedding(x)
        
        # 2. Position embeddings
        # Créer les positions [0, 1, 2, ..., S-1]
        positions = None  # torch.arange(S, device=x.device)
        pos_emb = None  # self.position_embedding(positions)
        
        # 3. Additionner et dropout
        x = None  # self.dropout(tok_emb + pos_emb)
        
        # 4. Passer par tous les blocs
        # for block in self.blocks:
        #     x = block(x)
        
        # 5. Layer norm finale
        x = None  # self.norm(x)
        
        # 6. Projection vers le vocabulaire
        logits = None  # self.head(x)
        
        return logits
    
    @torch.no_grad()
    def generate(self, context, max_new_tokens, temperature=1.0):
        """
        Génère du texte de manière autoregressive.
        
        Args:
            context: Tensor de shape (1, seq_len) avec le contexte initial
            max_new_tokens: Nombre de tokens à générer
            temperature: Contrôle la "créativité" (1.0 = normal, <1 = conservateur, >1 = créatif)
        
        Returns:
            Tensor avec le contexte + tokens générés
        """
        self.eval()
        
        for _ in range(max_new_tokens):
            # Tronquer si nécessaire
            context_truncated = context[:, -self.max_len:]
            
            # Forward
            logits = self(context_truncated)
            
            # Prendre les logits du dernier token
            logits = logits[:, -1, :] / temperature
            
            # Échantillonner
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Ajouter au contexte
            context = torch.cat([context, next_token], dim=1)
        
        return context

In [ ]:
# Test rapide
model_test = MiniGPT(vocab_size=100, embed_dim=64, num_heads=4, num_layers=2)
x_test = torch.randint(0, 100, (2, 32))
out_test = model_test(x_test)
print(f"Input: {x_test.shape}")
print(f"Output: {out_test.shape}")  # Attendu: (2, 32, 100)

---

## 4. Préparation des Données

On va entraîner notre Mini-GPT sur un corpus de texte français.

In [ ]:
# Télécharger un texte français (Les Misérables - Victor Hugo)
import urllib.request

url = "https://www.gutenberg.org/cache/epub/135/pg135.txt"
print("Téléchargement du corpus...")

with urllib.request.urlopen(url) as response:
    text = response.read().decode('utf-8')

# Nettoyer (garder une portion pour l'entraînement rapide)
# Le texte commence après les en-têtes Gutenberg
start_marker = "PREMIÈRE PARTIE"
start_idx = text.find(start_marker)
if start_idx != -1:
    text = text[start_idx:]

# Limiter la taille pour Colab (environ 500Ko)
text = text[:500000]

print(f"Taille du corpus: {len(text):,} caractères")
print(f"\nExtrait:\n{text[:500]}")

In [ ]:
# Créer le vocabulaire (niveau caractère)
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Vocabulaire: {vocab_size} caractères uniques")
print(f"Caractères: {''.join(chars[:50])}...")

# Mappings
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for i, ch in enumerate(chars)}

# Fonctions d'encodage/décodage
def encode(s):
    return [char2idx[c] for c in s]

def decode(indices):
    return ''.join([idx2char[i] for i in indices])

# Test
test_str = "Bonjour!"
encoded = encode(test_str)
decoded = decode(encoded)
print(f"\nTest: '{test_str}' -> {encoded} -> '{decoded}'")

In [ ]:
# Encoder tout le texte
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Data shape: {data.shape}")

# Split train/val (90/10)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(f"Train: {len(train_data):,} tokens")
print(f"Val: {len(val_data):,} tokens")

In [ ]:
# Dataset
class TextDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size
    
    def __len__(self):
        return len(self.data) - self.block_size
    
    def __getitem__(self, idx):
        # x = contexte, y = contexte décalé de 1 (ce qu'on veut prédire)
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

# Configuration
BLOCK_SIZE = 128  # Longueur des séquences
BATCH_SIZE = 64

train_dataset = TextDataset(train_data, BLOCK_SIZE)
val_dataset = TextDataset(val_data, BLOCK_SIZE)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print(f"Batches train: {len(train_loader)}")
print(f"Batches val: {len(val_loader)}")

In [ ]:
# Vérifier un batch
x_batch, y_batch = next(iter(train_loader))
print(f"x shape: {x_batch.shape}")  # (batch, block_size)
print(f"y shape: {y_batch.shape}")  # (batch, block_size)

# Exemple
print(f"\nExemple (input):  '{decode(x_batch[0].tolist())[:50]}...'")
print(f"Exemple (target): '{decode(y_batch[0].tolist())[:50]}...'")

---

## 5. Entraînement

In [ ]:
# Créer le modèle
model = MiniGPT(
    vocab_size=vocab_size,
    embed_dim=128,
    num_heads=4,
    num_layers=4,
    max_len=BLOCK_SIZE,
    dropout=0.1
).to(device)

# Compter les paramètres
num_params = sum(p.numel() for p in model.parameters())
print(f"Paramètres: {num_params:,}")

In [ ]:
# ============================================
# EXERCICE 3 : Boucle d'entraînement
# ============================================

def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    
    for x, y in tqdm(loader, desc="Training", leave=False):
        x, y = x.to(device), y.to(device)
        
        # TODO: Implémenter
        # 1. Zero grad
        optimizer.zero_grad()
        
        # 2. Forward
        logits = model(x)  # (batch, seq_len, vocab_size)
        
        # 3. Loss (cross-entropy)
        # Attention: reshape pour cross_entropy
        # logits: (B, S, V) -> (B*S, V)
        # y: (B, S) -> (B*S,)
        B, S, V = logits.shape
        loss = F.cross_entropy(logits.view(B*S, V), y.view(B*S))
        
        # 4. Backward
        loss.backward()
        
        # 5. Gradient clipping (stabilité)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 6. Optimizer step
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)


@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        B, S, V = logits.shape
        loss = F.cross_entropy(logits.view(B*S, V), y.view(B*S))
        total_loss += loss.item()
    
    return total_loss / len(loader)

In [ ]:
# Configuration
EPOCHS = 5
LR = 3e-4

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# Historique
history = {'train_loss': [], 'val_loss': []}

print("Début de l'entraînement...\n")

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = evaluate(model, val_loader, device)
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    # Générer un échantillon
    context = torch.tensor([encode("Jean Valjean ")], dtype=torch.long, device=device)
    generated = model.generate(context, max_new_tokens=100, temperature=0.8)
    print(f"  Génération: {decode(generated[0].tolist())[:100]}...")
    print()

In [ ]:
# Visualisation
plt.figure(figsize=(10, 4))
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Courbe d\'apprentissage')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## 6. Génération de Texte

In [ ]:
# ============================================
# EXERCICE 4 : Expérimenter avec la génération
# ============================================

def generate_text(prompt, max_tokens=200, temperature=1.0):
    """
    Génère du texte à partir d'un prompt.
    
    Args:
        prompt: Texte de départ
        max_tokens: Nombre de caractères à générer
        temperature: Créativité (0.5=conservateur, 1.0=normal, 1.5=créatif)
    """
    context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
    generated = model.generate(context, max_new_tokens=max_tokens, temperature=temperature)
    return decode(generated[0].tolist())

# Tests avec différentes températures
prompt = "La nuit était "

print("=" * 60)
print(f"Prompt: '{prompt}'")
print("=" * 60)

for temp in [0.5, 1.0, 1.5]:
    print(f"\n--- Température = {temp} ---")
    print(generate_text(prompt, max_tokens=150, temperature=temp))
    print()

In [ ]:
# Génération interactive
prompts = [
    "Il marchait dans la rue ",
    "L'évêque dit à ",
    "Paris est une ville ",
    "Le soleil se levait sur ",
]

for prompt in prompts:
    print(f"\n{'='*60}")
    print(f"Prompt: '{prompt}'")
    print(f"{'='*60}")
    print(generate_text(prompt, max_tokens=200, temperature=0.8))

---

## 7. Analyse et Visualisation

In [ ]:
# Visualiser les probabilités de prédiction
def visualize_predictions(text):
    """Visualise les probabilités de prédiction pour chaque caractère."""
    model.eval()
    
    x = torch.tensor([encode(text)], dtype=torch.long, device=device)
    
    with torch.no_grad():
        logits = model(x)
        probs = F.softmax(logits, dim=-1)
    
    # Pour chaque position, afficher le caractère prédit
    print(f"Texte: '{text}'\n")
    print("Position | Réel | Prédit | Confiance")
    print("-" * 45)
    
    for i in range(len(text) - 1):
        actual = text[i + 1]
        pred_idx = probs[0, i].argmax().item()
        pred_char = idx2char[pred_idx]
        confidence = probs[0, i, char2idx[actual]].item()
        
        match = "✓" if pred_char == actual else "✗"
        actual_display = repr(actual)[1:-1]  # Afficher les caractères spéciaux
        pred_display = repr(pred_char)[1:-1]
        
        print(f"   {i:3d}   | '{actual_display:2s}' | '{pred_display:2s}'  | {confidence:.2%} {match}")

visualize_predictions("Jean Valjean")

---

## 8. Récapitulatif

### Ce que vous avez appris

1. **Différence Encodeur/Décodeur** : Le masque causal empêche de "voir le futur"
2. **Architecture GPT** : Attention causale + génération autoregressive
3. **Entraînement** : Prédire le caractère suivant (next token prediction)
4. **Température** : Contrôle le compromis entre cohérence et créativité

### Comparaison avec les vrais GPT

| | Notre Mini-GPT | GPT-2 Small | GPT-3 |
|--|----------------|-------------|-------|
| Paramètres | ~1M | 117M | 175B |
| Données | ~500Ko | 40Go | ~500Go |
| Vocabulaire | Caractères | BPE (~50k) | BPE (~50k) |
| Contexte | 128 | 1024 | 2048 |

### Limitations de notre modèle

- **Vocabulaire caractère** : Inefficace, perd la notion de "mot"
- **Petit corpus** : Surapprentissage probable
- **Contexte court** : Ne peut pas capturer les dépendances longues

### Pour aller plus loin

- Utiliser un tokenizer BPE (comme SentencePiece)
- Entraîner sur plus de données
- Ajouter des techniques modernes (RoPE, KV-cache, Flash Attention)

In [ ]:
# Espace pour vos expérimentations

# Essayez différents prompts !
mon_prompt = "Cosette regardait "
print(generate_text(mon_prompt, max_tokens=300, temperature=0.8))